In [1]:
import tensorflow as tf
import pandas as pd
import json
import re

loads -> json strings

load -> json file

In [3]:
try:
  df=pd.read_json('dataset.json')
except ValueError:
  with open('dataset.json','r') as file:
    lis=[]
    for line in file:
      if(line.strip()):
        lis.append(json.loads(line))
  df=pd.DataFrame(lis)

In [4]:
df.head()

,plain_text,summary
0,We can change these Terms at any time. We keep...,"Users should revisit the terms periodically, a..."
1,How To File a DMCA Notice To submit a notice o...,This service will aid you when other users inf...
2,You can see our previous Privacy Policy her...,There is a date of the last update of the agre...
3,One concrete way we commit to user privacy is ...,This service respects your browser's Do Not Tr...
4,The categories of information we collect can i...,The service provides details about what kinds ...


In [5]:
df_new=pd.DataFrame(columns=df.columns)
df_new.head()

,plain_text,summary


In [6]:
j=0
for i in range(len(df)):
  l=len(df.iloc[i,0])
  if(l<=512):
    df_new.loc[j]=df.loc[i]
    j+=1

In [7]:
df=df_new

In [8]:
df.isnull().sum()

,0
plain_text,0
summary,0


In [9]:
df.shape

(127, 2)

In [10]:
def clean_text(text):

  text=re.sub(r'\s+', ' ', text)
  text=text.strip()
  return text

df['plain_text']=df['plain_text'].apply(clean_text)

In [13]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

# Load the pre-trained model and tokenizer
pretrained_model = "t5-base"  # You can choose any model size you want, like t5-small, t5-base, etc.
tokenizer = T5Tokenizer.from_pretrained(pretrained_model, legacy=False)

# Use the TensorFlow version of T5 model
model = TFT5ForConditionalGeneration.from_pretrained(pretrained_model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [14]:
df.columns

Index(['plain_text', 'summary'], dtype='object')

In [15]:
converted_sentences=tokenizer(df['plain_text'].to_list(),padding='max_length',max_length=512,return_tensors='tf',truncation=True)

In [16]:
converted_summaries=tokenizer(df['summary'].to_list(),padding='max_length',max_length=512,return_tensors='tf',truncation=True)

In [17]:
converted_summaries

{'input_ids': <tf.Tensor: shape=(127, 512), dtype=int32, numpy=
array([[  100,   313,  1445, ...,     0,     0,     0],
       [  100,   313,  2311, ...,     0,     0,     0],
       [13504,  2065,    59, ...,     0,     0,     0],
       ...,
       [13504,  2065,    59, ...,     0,     0,     0],
       [  148,  1961,  7915, ...,     0,     0,     0],
       [    3,  3626,     8, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(127, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [18]:
converted_sentences

{'input_ids': <tf.Tensor: shape=(127, 512), dtype=int32, numpy=
array([[  555,  4427,   194, ...,     0,     0,     0],
       [ 2847, 13458, 11973, ...,     0,     0,     0],
       [  933,   169,     8, ...,     0,     0,     0],
       ...,
       [  861,  5569,    29, ...,     0,     0,     0],
       [  148,    56,  7365, ...,     0,     0,     0],
       [ 9428,   270,   225, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(127, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

Preparing a tensor dataset

In [23]:
# Preparing a tensor dataset
labels = converted_summaries['input_ids']

train_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {
            'input_ids': converted_sentences['input_ids'],
            'attention_mask': converted_sentences['attention_mask'],
            'decoder_input_ids': converted_summaries['input_ids'],
        },
        labels
    )
)

train_dataset = train_dataset.shuffle(buffer_size=100).batch(2)

In [24]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [25]:
model.fit(train_dataset,epochs=5)

Epoch 1/5
64/64 [==============================] - 103s 640ms/step - loss: 5.1964 - accuracy: 0.4387
Epoch 2/5
64/64 [==============================] - 43s 665ms/step - loss: 0.5799 - accuracy: 0.9112
Epoch 3/5
64/64 [==============================] - 42s 662ms/step - loss: 0.4044 - accuracy: 0.9340
Epoch 4/5
64/64 [==============================] - 42s 661ms/step - loss: 0.3484 - accuracy: 0.9404
Epoch 5/5
64/64 [==============================] - 43s 664ms/step - loss: 0.3038 - accuracy: 0.9465


In [57]:
prediction_text="""As AI becomes more pervasive, we can expect it to take on mundane and tedious tasks that require a lot of human effort and time. One such activity is legal reviews of SaaS contracts between vendors and enterprise companies. Lawyers are quite expensive (ranging from 500$-1000$ an hour) and it would be great to have an AI agent conduct legal review of contracts and flag the risks to the buyer of SaaS within minutes. As part of the jazzee marketplace, when the buyer is about to purchase a product, he/she needs to sign off the Terms and Conditions. We need the AI conversational bot Janie to quickly scan the T&C and outline any risks back to the buyer so he/she can click OK and move on with the purchase or push it back to the vendor for changes"""

In [58]:
tokenized_text=tokenizer(prediction_text,max_length=500,padding='max_length',truncation=True,return_tensors='tf')

In [59]:
tokenized_text

{'input_ids': <tf.Tensor: shape=(1, 500), dtype=int32, numpy=
array([[  282,  7833,  2992,    72,   399,  9856,   757,     6,    62,
           54,  1672,    34,    12,   240,    30, 29826,    11, 28043,
         4145,    24,  1457,     3,     9,   418,    13,   936,  1941,
           11,    97,     5,   555,   224,  1756,    19,  1281,  2456,
           13,  1138,     9,   134,  8201,   344,  8535,    11,  5399,
          688,     5, 18768,     7,    33,   882,  2881,    41,  6836,
           45,  2899,  3229,    18, 16824,  3229,    46,  1781,    61,
           11,    34,   133,    36,   248,    12,    43,    46,  7833,
         3102,  3498,  1281,  1132,    13,  8201,    11,  5692,     8,
         5217,    12,     8,  8001,    13,  1138,     9,   134,   441,
          676,     5,   282,   294,    13,     8,  9948,    15,    15,
         9895,     6,   116,     8,  8001,    19,    81,    12,  1242,
            3,     9,   556,     6,     3,    88,    87,     7,    88,
          523, 

In [62]:
summary=model.generate(tokenized_text['input_ids'],max_length=1000, num_beams=4)
summary

<tf.Tensor: shape=(1, 115), dtype=int32, numpy=
array([[    0, 32099,  9196,    33,   882,  2881,    41,  6836,    45,
         2899,  3229,    18, 16824,  3229,    46,  1781,    61,    11,
           34,   133,    36,   248,    12,    43,    46,  7833,  3102,
         3498,  1281,  2456,    13,  1138,     9,   134,  8201,   344,
         8535,    11,  5399,   688,     5,   101,   174,     8,  7833,
         3634,   138, 14761,  3049,    23,    15,    12,  1224,  5924,
            8,   332,   184,   254,    11, 11052,   136,  5217,    12,
            8,  8001,    13,  1138,     9,   134,     5,   101,   174,
            8,  7833,  3634,   138, 14761,  3049,    23,    15,    12,
         1224,  5924,     8,   332,   184,   254,    11, 11052,   136,
         5217,    12,     8,  8001,    13,  1138,     9,   134,     5,
            1,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>

In [63]:
tokenizer.decode(summary[0],skip_special_tokens=True)

'lawyers are quite expensive (ranging from 500$-1000$ an hour) and it would be great to have an AI agent conduct legal reviews of SaaS contracts between vendors and enterprise companies. We need the AI conversational bot Janie to quickly scan the T&C and outline any risks to the buyer of SaaS. We need the AI conversational bot Janie to quickly scan the T&C and outline any risks to the buyer of SaaS.'

In [69]:
model.save_pretrained("fine_tuned_model")

In [72]:
# Save the model and tokenizer to Google Drive
model_save_path = '/content/drive/My Drive/my_model'
tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)